In [ ]:
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
data = pd.read_csv("/content/fmnist_small.csv")

In [ ]:
data.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,...,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,125,72,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,3,0,2,0,0,210,228,228,233,0,0,0,0,0,0,0,0,0,31,81,133,184,201,190,117,0,0,2,1,0,7,0,50,205,196,213,165,0,0
1,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,0,43,117,34,15,24,33,117,80,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,51,144,152,...,157,158,161,148,159,58,0,6,0,0,0,0,0,0,0,0,0,4,0,60,143,143,148,146,152,152,148,148,147,145,142,142,142,21,0,3,0,0,0,0
3,8,0,0,0,0,0,0,0,0,0,0,0,0,2,0,33,114,37,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,11,17,13,13,11,10,7,5,5,5,7,0,0,0,1,0,0,41,69,88,86,94,106,114,118,47,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,8,0,0,0,0,0,0,0,0,0,0,2,0,58,145,114,10,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,...,165,153,155,134,143,172,215,62,0,0,0,0,0,0,0,0,10,190,178,194,209,211,209,205,211,215,213,217,225,228,213,203,174,151,188,10,0,0,0,0


In [ ]:
X = data.iloc[:,1:].values
y= data.iloc[:,0].values
print(y)

[9 7 0 ... 8 4 8]


In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=143)


In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
from torch.utils.data import Dataset,DataLoader
class dataset(Dataset):

  def __init__(self,feature,label):
    self.feature = torch.tensor(feature,dtype=torch.float32)
    self.label = torch.tensor(label,dtype=torch.long)

  def __len__(self):
    return len(self.feature)

  def __getitem__(self,index):
    return self.feature[index],self.label[index]

In [ ]:
y_test.shape

(1200,)

In [ ]:
train_data = dataset(X_train,y_train)
test_data = dataset(X_test,y_test)

In [ ]:
train_loader = DataLoader(train_data,batch_size=32,shuffle=True)
test_loader = DataLoader(test_data,batch_size=32,shuffle=False)

In [ ]:
X_train.shape

(4800, 784)

In [ ]:
!pip3 install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 18.1 MB/s eta 0:00:00


In [ ]:
class Mymodel(nn.Module):

  def __init__(self,input_dim,output_dim,hidden_layers,num_neurons,drop_ratio):

    super().__init__()
    layers=[]
    for i in range(hidden_layers):
      layers.append(nn.Linear(input_dim,num_neurons))
      layers.append(nn.BatchNorm1d(num_neurons))
      layers.append(nn.ReLU())
      layers.append(nn.Dropout(drop_ratio))
      input_dim = num_neurons
    layers.append(nn.Linear(num_neurons,output_dim))
    self.model = nn.Sequential(*layers)

  def forward(self,x):
    return self.model(x)

In [ ]:
def objective_function(trial):
  hidden_layers = trial.suggest_int('hidden_layers',1,5)
  num_neurons = trial.suggest_int('num_neurons',8,128,step=8)
  drop_ratio = trial.suggest_float('drop_ratio',0.1,0.5,step=0.1)
  epochs = trial.suggest_int('epochs',10,100,step=10)
  learning_rate = trial.suggest_float('learning_rate',0.01,0.1,step=0.1)

  optimizer_name = trial.suggest_categorical('optimizer_name',['Adam','SGD','RMS'])
  batch_size = trial.suggest_categorical('batch_size',[16,32,64,128])

  train_loader = DataLoader(train_data,batch_size=batch_size,shuffle=True)
  test_loader = DataLoader(test_data,batch_size=batch_size,shuffle=False)

  input_dim=784
  output_dim=10
  model = Mymodel(input_dim,output_dim,hidden_layers,num_neurons,drop_ratio)
  learning_rate = learning_rate
  epochs=epochs

  if optimizer_name=='Adam':
    optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)
  elif optimizer_name=='SGD':
    optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)
  else:
    optimizer = torch.optim.RMSprop(model.parameters(),lr=learning_rate)

  loss_func = nn.CrossEntropyLoss()

  # traing loop

  for i in range(epochs):
    for batch_feature,batch_label in train_loader:

      y_pred = model(batch_feature)

      loss = loss_func(y_pred,batch_label)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
  # evaluation
  total_count=0
  correct=0
  for batch_feature,batch_label in test_loader:

    outputs = model(batch_feature)

    output = torch.max(outputs,1)[1]
    total_count+=batch_feature.shape[0]

    correct+=(output==batch_label).sum().item()
  accuracy =correct/total_count

  return accuracy


In [ ]:
import optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective_function,n_trials = 10)

[I 2025-07-08 07:05:36,413] A new study created in memory with name: no-name-e4f91fc9-d074-402d-a0ab-2694a161803b
[I 2025-07-08 07:06:04,904] Trial 0 finished with value: 0.7975 and parameters: {'hidden_layers': 1, 'num_neurons': 48, 'drop_ratio': 0.30000000000000004, 'epochs': 90, 'learning_rate': 0.01, 'optimizer_name': 'Adam', 'batch_size': 32}. Best is trial 0 with value: 0.7975.
[I 2025-07-08 07:08:06,883] Trial 1 finished with value: 0.7883333333333333 and parameters: {'hidden_layers': 5, 'num_neurons': 64, 'drop_ratio': 0.1, 'epochs': 90, 'learning_rate': 0.01, 'optimizer_name': 'Adam', 'batch_size': 16}. Best is trial 0 with value: 0.7975.
[I 2025-07-08 07:08:21,411] Trial 2 finished with value: 0.44666666666666666 and parameters: {'hidden_layers': 5, 'num_neurons': 8, 'drop_ratio': 0.2, 'epochs': 70, 'learning_rate': 0.01, 'optimizer_name': 'Adam', 'batch_size': 128}. Best is trial 0 with value: 0.7975.
[I 2025-07-08 07:08:48,537] Trial 3 finished with value: 0.7525 and parame

In [ ]:
study.best_value

0.8033333333333333

In [ ]:
study.best_params

{'hidden_layers': 5,
 'num_neurons': 56,
 'drop_ratio': 0.1,
 'epochs': 70,
 'learning_rate': 0.01,
 'optimizer_name': 'Adam',
 'batch_size': 32}